In [2]:
%pip install ortools scikit-learn

import time
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
from ortools.constraint_solver import routing_enums_pb2, pywrapcp

# 1) HARD-CODED “REAL” DISTANCES for Bengaluru DC → 8 retail zones (in meters)
distance_matrix = [
    [    0,  7000,  9000, 10000, 16000, 20000,  8000, 12000, 14000],
    [ 7000,     0,  5000, 10000, 20000, 10000, 15000, 14000, 16000],
    [ 9000,  5000,     0, 12000, 18000, 15000, 12000, 13000, 15000],
    [10000, 10000, 12000,     0,  8000, 17000, 12000, 14000, 15000],
    [16000, 20000, 18000,  8000,     0, 30000, 20000, 18000, 16000],
    [20000, 10000, 15000, 17000, 30000,     0, 18000, 22000, 24000],
    [ 8000, 15000, 12000, 12000, 20000, 18000,     0,  4000,  6000],
    [12000, 14000, 13000, 14000, 18000, 22000,  4000,     0,  7000],
    [14000, 16000, 15000, 15000, 16000, 24000,  6000,  7000,     0],
]

# 2) FMCG DEMANDS & INITIAL FLEET INFO
data = {
    'distance_matrix': distance_matrix,
    'num_vehicles':    4,
    'depot':           0,
    'demands':   [0, 200,150,180,220,250,170,190,160],
    # (we’ll overwrite these with ML predictions)
    'vehicle_capacities': [800,700,1000,600],
}

# 3) SETUP THE OR-TOOLS SOLVER (unchanged)
search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_params.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_params.time_limit.FromSeconds(3)

def solve_vrp(capacities, label):
    data['vehicle_capacities'] = capacities
    mgr = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot'])
    routing = pywrapcp.RoutingModel(mgr)

    # DISTANCE CALLBACK
    def dist_cb(i, j):
        return data['distance_matrix'][mgr.IndexToNode(i)][mgr.IndexToNode(j)]
    tidx = routing.RegisterTransitCallback(dist_cb)
    routing.SetArcCostEvaluatorOfAllVehicles(tidx)

    # DEMAND & CAPACITY
    def demand_cb(i):
        return data['demands'][mgr.IndexToNode(i)]
    didx = routing.RegisterUnaryTransitCallback(demand_cb)
    routing.AddDimensionWithVehicleCapacity(didx, 0,
                                           capacities,
                                           True, 'Capacity')

    sol = routing.SolveWithParameters(search_params)
    print(f"\n[{label}] capacities → {capacities}")
    if not sol:
        print(" ⚠️ No feasible solution!")
        return

    total_d, total_l = 0, 0
    for v in range(data['num_vehicles']):
        idx, route, load, dist = routing.Start(v), [], 0, 0
        while not routing.IsEnd(idx):
            n = mgr.IndexToNode(idx)
            load += data['demands'][n]
            route.append((n, load))
            p = idx
            idx = sol.Value(routing.NextVar(idx))
            dist += routing.GetArcCostForVehicle(p, idx, v)
        route.append((mgr.IndexToNode(idx), load))
        print(f"  ▶️ Vehicle {v}: {route} | dist={dist}m load={load}b")
        total_d += dist
        total_l += load

    print(f"🚚 Fleet total: {total_d}m, delivered {total_l}/{sum(data['demands'])} boxes")


# 4) TRAIN A SIMPLE ML MODEL TO PREDICT CAPACITY BY HOUR-OF-DAY
#    (Replace these arrays with your historical load/unload logs!)
hours_hist = np.array([[8], [9], [10], [11], [12], [13], [14], [15], [16]] )
caps_hist   = np.array([
    [780, 680, 980, 580],  # 08:00
    [760, 700,1000,600],   # 09:00
    [740, 720, 960,620],   # …
    [720, 710, 940,610],
    [700, 730, 920,630],
    [680, 740, 900,640],
    [700, 720, 940,620],
    [720, 700, 960,600],
    [740, 680, 980,580],
])
models = [LinearRegression().fit(hours_hist, caps_hist[:,i])
          for i in range(data['num_vehicles'])]

def predict_capacities(current_time):
    h = np.array([[ current_time.hour + current_time.minute/60 ]])
    return [ int(m.predict(h)[0]) for m in models ]


# 5) RUN A “REAL-TIME” LOOP (every 5 minutes)
#    — in production, trigger on actual telemetry/capacity events instead of sleep ---
if __name__ == '__main__':
    try:
        while True:
            now   = datetime.now()
            caps  = predict_capacities(now)
            label = now.strftime('%H:%M')
            solve_vrp(caps, label)
            time.sleep(5*60)   # wait 5 min
    except KeyboardInterrupt:
        print("\n🛑 Stopped real-time re-optimization.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0

[06:44] capacities → [761, 706, 970, 606]
  ▶️ Vehicle 0: [(0, 0), (0, 0)] | dist=0m load=0b
  ▶️ Vehicle 1: [(0, 0), (1, 200), (5, 450), (2, 600), (0, 600)] | dist=41000m load=600b
  ▶️ Vehicle 2: [(0, 0), (3, 180), (4, 400), (8, 560), (7, 750), (6, 920), (0, 920)] | dist=53000m load=920b
  ▶️ Vehicle 3: [(0, 0), (0, 0)] | dist=0m load=0b
🚚 Fleet total: 94000m, delivered 1520/1520 boxes

🛑 Stopped real-time re-optimization.
